## Load Base Stable Diffusion Model

In [ ]:
from diffusers import StableDiffusionPipeline
from pruna.algorithms.smasher_config import AutoSmasherConfig
from pruna.smash import smash
import torch
import requests
from PIL import Image
from io import BytesIO

from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image
import torch

pipe = AutoPipelineForInpainting.from_pretrained("diffusers/stable-diffusion-xl-1.0-inpainting-0.1", torch_dtype=torch.float16, variant="fp16").to("cuda")

img_url = "https://raw.githubusercontent.com/CompVis/latent-diffusion/main/data/inpainting_examples/overture-creations-5sI6fQgYIuo.png"
mask_url = "https://raw.githubusercontent.com/CompVis/latent-diffusion/main/data/inpainting_examples/overture-creations-5sI6fQgYIuo_mask.png"

image = load_image(img_url).resize((1024, 1024))
mask_image = load_image(mask_url).resize((1024, 1024))

prompt = "a tiger sitting on a park bench"
generator = torch.Generator(device="cuda").manual_seed(0)

## Smash it!

In [ ]:
from pruna.algorithms.smasher_config import SmasherConfig

smasher_config = SmasherConfig()
smasher_config['compiler'] = 'diffusers'
smasher_config['max_batch_size'] = 1
smasher_config['image_height'] = 1024
smasher_config['image_width'] = 1024
smasher_config['version'] = 'xl-1.0'
smasher_config['static_shape'] = False
smasher_config['unet_dim'] = 9

In [ ]:
smashed_model = smash(
        model=pipe,
        data_module="inpaint",
        api_key='your-api-key',
        model_config=None,
        smasher_config=smasher_config,
        device='cuda',
    )

## Base Model Generation

In [ ]:
%%time
pipe(prompt=prompt, image=image, mask_image=mask_image).images[0].show()

## Smashed Model Generation

In [ ]:
%%time
smashed_model(prompt, input_image=image, mask_image=mask_image, num_inference_steps=50)[0]